## Stream / Voice Expression

This notebook uses the streaming API to analyze a stream of vocal audio data.

In [1]:
import asyncio
import traceback

from utilities import download_file, encode_audio, generate_audio_stream, print_emotions

from hume import HumeStreamClient
from hume.models.config import BurstConfig, ProsodyConfig

In [ ]:
!pip install sounddevice numpy scipy pydub


In [2]:
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write
from pydub import AudioSegment

def record_audio(duration=5, fs=44100):
    """
    Record audio from the microphone.

    Parameters:
    - duration: Length of the recording in seconds.
    - fs: Sampling rate.

    Returns:
    - Numpy array containing the recorded audio data.
    """
    print(f"Recording for {duration} seconds...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=2, dtype='float64')
    sd.wait()  # Wait until recording is finished
    print("Recording finished")
    return recording, fs

def save_as_mp3(audio_data, fs, filename='recording.mp3'):
    """
    Save the audio data to an MP3 file.

    Parameters:
    - audio_data: Numpy array containing the audio data.
    - fs: Sampling rate.
    - filename: Filename for the MP3 file.
    """
    # Convert the Numpy array to a WAV file temporarily
    temp_wav = "temp_recording.wav"
    write(temp_wav, fs, audio_data)

    # Convert WAV to MP3
    sound = AudioSegment.from_wav(temp_wav)
    sound.export(filename, format="mp3")
    
    # Remove temporary WAV file
    import os
    os.remove(temp_wav)

    print(f"Saved recording as {filename}")

# Example usage
audio_data, fs = record_audio(duration=5)  # Record for 5 seconds
save_as_mp3(audio_data, fs, 'my_recording.mp3')


OSError: PortAudio library not found

In [ ]:
filepath = download_file("https://storage.googleapis.com/hume-test-data/audio/ninth-century-laugh.mp3")

async def main():
    try:
        client = HumeStreamClient("3zSBNpCCtON0nUM6VDiqd9pD8Zi4SHRqnvOYtIjkBG13jKWE")
        burst_config = BurstConfig()
        prosody_config = ProsodyConfig()
        async with client.connect([burst_config, prosody_config]) as socket:

            for sample_number, audio_sample in enumerate(generate_audio_stream(filepath)):
                encoded_sample = encode_audio(audio_sample)
                
                # Reset connection stream context between samples
                await socket.reset_stream()

                result = await socket.send_bytes(encoded_sample)

                print(f"\nStreaming sample {sample_number + 1}")

                print("Speech prosody:")
                if "warning" in result["prosody"]:
                    print(result["prosody"]["warning"])
                else:
                    emotions = result["prosody"]["predictions"][0]["emotions"]
                    print_emotions(emotions)

                print("Vocal burst")
                if "warning" in result["burst"]:
                    print(result["burst"]["warning"])
                else:
                    emotions = result["burst"]["predictions"][0]["emotions"]
                    print_emotions(emotions)
    except Exception:
        print(traceback.format_exc())

# When running the streaming API outside of a Jupyter notebook you do not need these lines.
# Jupyter has its own async event loop, so this merges main into the Jupyter event loop.
# To run this sample in a script with asyncio you can use `asyncio.run(main())`
loop = asyncio.get_event_loop()
loop.create_task(main())